In [5]:
%run load_helper.ipynb

# ##################################################
#             SET PARAMETERS
#
# pick a baseline
#        baselines = ["10topics", "15topics", "20topics", "25topics", "30topics"]
#
# name your run (used to store the iterations); models will be saved myrun1.sav myrun2.save etc
#
# set your parameters
# don't forget to set the number of topics for that baseline
#
# ##################################################
runname = "initialrun"
mybaseline = "20topicsv"
num_topics = 20 # set this based upon baseline 
                # based on our analysis of coherence, 20 is a good starting number

# below are default params that can be played with
num_buffers = 5    # how many buffers to add each iteration
lda_decay = .5     # how much the prior influences the iteration 0 - 1 
num_iterations = 1 # the article used 5


# ##################################################
#
# The ITMFT algorithm
#
#  Run cell 1
#  and either cell 2 (to create a baseline)
#      or cell 3 to load the baseline
# ##################################################
file_name = save_path + mybaseline + ".sav"
model = LdaModel.load(file_name)

iteration = 0
oldsignificanttopics = 0
significanttopcs = 1

while iteration < num_iterations and significanttopcs > oldsignificanttopics :
    # create a topic coverage matrix preset to 0
    topiccoverage = []
    i = 0
    while i < len(docs_per_timeslice) :
        y = 0
        thistopic = []
        while y < num_topics:
            thistopic.append(0.0)
            y += 1
        topiccoverage.append(thistopic)
        i += 1
            
    # get the topic coverage per timeslice per doc
    timeslice = 0
    for timeslicedocs in docs_per_timeslice :
        # for each doc in this timeslice
        for doc in timeslicedocs :
            # get the probability matrix
            probs = model.get_document_topics(bow[doc])
            #its a sparse array, prob[0] is the topic and prob[1] is the probabiltiy
            for prob in probs :
                topiccoverage[timeslice][prob[0]] += prob[1]
        timeslice += 1
        
    topics = model.get_topics()
    
    newtopics = []
    # ##################################################
    # INPUT:
    # timeslicetokencounts - we have the word coverage for each model token
    # topiccoverage - and now we have the topic coverage
    # topics - the current topic word probabilities
    #
    # OUTPUT:
    # updated array newtopics = []
    #
    # run the iteration
    # ##################################################
    
    #%run ITMTF_iterate.ipynb

    
    # ##################################################
    #
    # using the returned topics probabilies
    # and adding buffers - num_buffers using bufferprob
    # create the prior
    # update num_topics
    # and run the model
    # ##################################################
    
    #$$$ remove this, for now just creating a topic prob list of X from old model as the return
    topics = model.get_topics()
    print(len(topics))
    for topic in topics :
        newtopics.append(topic)
    #$$$ remove this, for now just creating a topic prob list of X from old model as the return
        
    # add the buffers
    z = 0
    while z < num_buffers:
        newtopics.append(bufferprob)
        z += 1
    num_topics = len(newtopics)

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Iteration start timee =", current_time)
    model = LdaModel(
        corpus=corpus,
        id2word=tokentoword,
        chunksize=chunksize,
        alpha='auto',               
        eta=newtopics,                 # preset topic/word
        iterations=iterations,
        num_topics=num_topics,         # added buffer topics
        passes=passes,
        decay = lda_decay,
        eval_every=eval_every
    )

    file_name = runname + str(iteration) 
    path_name = save_path + file_name + ".sav"
    print(file_name + " - saved for visualization")
    model.save(path_name )
 
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Iteration Complete = ", current_time)

    iteration += 1
print("Run Complete ")


Number of time slices with docs: 123
Number of time slices: 123
Number of time vocab: 12517
Number of documents: 2673
Number of unique tokens: 12517
20
Iteration start timee = 14:56:40
initialrun0.sav saved for visualization
Iteration Complete =  15:04:04
Run Complete 
